# 6 Nations Rugby match Predictor

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
le = LabelEncoder()
ohe=OneHotEncoder(handle_unknown='ignore')

In [3]:
#Lecture du dataset
df = pd.read_csv(r'/data/HomeData.csv')
df.head()

,Team,Diff,Opposition,Ground,Match Date
0,Wales,8,v Italy,Millennium Stadium,1 Feb 2014
1,France,2,v England,Stade de France,1 Feb 2014
2,France,-5,v Wales,Stade de France,1 Feb 2019
3,Ireland,7,v Scotland,Lansdowne Road,1 Feb 2020
4,Wales,42,v Italy,Millennium Stadium,1 Feb 2020


## NETTOYAGE DU DATASET (PREPROCESSING)

In [4]:
year = []
opponent = []
for index, row in df.iterrows():
    date = row["Match Date"]  #On garde que l'année dans la date
    year.append(date[-4:])
for index, row in df.iterrows():
    opp = row["Opposition"]  #On enlève le v
    opponent.append(opp[2:])
df["Opponent"] = opponent
df["Year"] = year

#On retire les colonnes qui ne nous intéressent pas
df = df.drop("Opposition", axis='columns')
df = df.drop("Match Date", axis='columns')
df = df.drop("Ground", axis='columns')
df = df[['Team', 'Opponent', 'Year', 'Diff']]
df.head()

,Team,Opponent,Year,Diff
0,Wales,Italy,2014,8
1,France,England,2014,2
2,France,Wales,2019,-5
3,Ireland,Scotland,2020,7
4,Wales,Italy,2020,42


AJOUT DU RATING SCORE PAR ANNÉE

In [5]:
rating = pd.read_csv(r'/data/RugbyStats.csv')
rating

,Member Union,Tier,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ENGLAND,1,High Performance,93.99,86.59,83.27,78.61,85.57,80.87,80.72,82.48,81.58,83.90,85.70,85.68,79.76,90.47,90.88,86.23,88.73
1,IRELAND,1,High Performance,80.53,82.61,80.03,84.67,78.67,78.51,84.60,81.79,80.65,80.22,79.33,82.13,81.17,84.63,86.40,91.18,84.24
2,WALES,1,High Performance,76.92,76.91,82.54,76.87,74.17,81.03,79.89,77.04,79.61,78.39,80.19,80.70,83.49,82.56,82.09,87.26,79.77
3,FRANCE,1,High Performance,82.71,83.70,86.10,85.87,80.36,79.51,82.41,81.66,84.70,85.07,80.87,80.07,79.77,80.14,78.11,77.34,85.66
4,SCOTLAND,1,High Performance,76.36,74.65,73.52,76.73,76.70,76.75,76.59,81.20,76.20,75.83,76.56,75.16,78.01,80.71,84.13,81.86,81.21
5,ITALY,1,High Performance,71.88,72.51,72.18,72.79,73.29,74.56,72.59,73.31,73.99,76.24,73.13,72.05,72.79,72.50,71.27,72.76,70.88


In [6]:
standing_home = []
standing_away = []

# function so given a team and a year I can locate the standing in the standing dataframe and return it.

def get_standing(team, year):
    number = 0
    if team == "England":
        number = 0
    elif team == "Ireland":
        number = 1
    elif team == "Wales":
        number = 2
    elif team == "France":
        number = 3
    elif team == "Scotland":
        number = 4
    else:
        number = 5
    standing = rating.at[number, year]
    return standing


for index, row in df.iterrows():
    standing_home.append(get_standing(row["Team"], row["Year"]))
    standing_away.append(get_standing(row["Opponent"], row["Year"]))

df["home_standing"] = standing_home
df["away_standing"] = standing_away

df

,Team,Opponent,Year,Diff,home_standing,away_standing
0,Wales,Italy,2014,8,80.19,73.13
1,France,England,2014,2,80.87,85.70
2,France,Wales,2019,-5,77.34,87.26
3,Ireland,Scotland,2020,7,84.24,81.21
4,Wales,Italy,2020,42,79.77,70.88
...,...,...,...,...,...,...
250,Ireland,France,2013,0,80.22,85.07
251,Scotland,Wales,2013,-10,75.83,78.39
252,England,Wales,2014,11,85.70,80.19
253,Scotland,Wales,2019,-7,81.86,87.26


ON TRANSFORME LES VALEURS CATÉGORIELLES EN VALEURS NUMÉRIQUES (ONE HOT ENCODER)

In [7]:
#One hot encoding de la colonne Team
num_team = pd.DataFrame(ohe.fit_transform(df[['Team']]).toarray())
df = df.join(num_team)
df.head()

,Team,Opponent,Year,Diff,home_standing,away_standing,0,1,2,3,4,5
0,Wales,Italy,2014,8,80.19,73.13,0.0,0.0,0.0,0.0,0.0,1.0
1,France,England,2014,2,80.87,85.70,0.0,1.0,0.0,0.0,0.0,0.0
2,France,Wales,2019,-5,77.34,87.26,0.0,1.0,0.0,0.0,0.0,0.0
3,Ireland,Scotland,2020,7,84.24,81.21,0.0,0.0,1.0,0.0,0.0,0.0
4,Wales,Italy,2020,42,79.77,70.88,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
#On renomme les colonnes transformées
df.rename(columns = {0:'Team_0'}, inplace = True)
df.rename(columns = {1:'Team_1'}, inplace = True)
df.rename(columns = {2:'Team_2'}, inplace = True)
df.rename(columns = {3:'Team_3'}, inplace = True)
df.rename(columns = {4:'Team_4'}, inplace = True)
df.rename(columns = {5:'Team_5'}, inplace = True)

df.head()

,Team,Opponent,Year,Diff,home_standing,away_standing,Team_0,Team_1,Team_2,Team_3,Team_4,Team_5
0,Wales,Italy,2014,8,80.19,73.13,0.0,0.0,0.0,0.0,0.0,1.0
1,France,England,2014,2,80.87,85.70,0.0,1.0,0.0,0.0,0.0,0.0
2,France,Wales,2019,-5,77.34,87.26,0.0,1.0,0.0,0.0,0.0,0.0
3,Ireland,Scotland,2020,7,84.24,81.21,0.0,0.0,1.0,0.0,0.0,0.0
4,Wales,Italy,2020,42,79.77,70.88,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
#One Hot Encoding de la colonne Opponent
num_opponent = pd.DataFrame(ohe.fit_transform(df[['Opponent']]).toarray())
df = df.join(num_opponent)
df.head()

,Team,Opponent,Year,Diff,home_standing,away_standing,Team_0,Team_1,Team_2,Team_3,Team_4,Team_5,0,1,2,3,4,5
0,Wales,Italy,2014,8,80.19,73.13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,France,England,2014,2,80.87,85.70,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,France,Wales,2019,-5,77.34,87.26,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Ireland,Scotland,2020,7,84.24,81.21,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Wales,Italy,2020,42,79.77,70.88,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
#On renomme les colonnes transformées
df.rename(columns = {0:'Opponent_0'}, inplace = True)
df.rename(columns = {1:'Opponent_1'}, inplace = True)
df.rename(columns = {2:'Opponent_2'}, inplace = True)
df.rename(columns = {3:'Opponent_3'}, inplace = True)
df.rename(columns = {4:'Opponent_4'}, inplace = True)
df.rename(columns = {5:'Opponent_5'}, inplace = True)

df

,Team,Opponent,Year,Diff,home_standing,away_standing,Team_0,Team_1,Team_2,Team_3,Team_4,Team_5,Opponent_0,Opponent_1,Opponent_2,Opponent_3,Opponent_4,Opponent_5
0,Wales,Italy,2014,8,80.19,73.13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,France,England,2014,2,80.87,85.70,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,France,Wales,2019,-5,77.34,87.26,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Ireland,Scotland,2020,7,84.24,81.21,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Wales,Italy,2020,42,79.77,70.88,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,Ireland,France,2013,0,80.22,85.07,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
251,Scotland,Wales,2013,-10,75.83,78.39,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
252,England,Wales,2014,11,85.70,80.19,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
253,Scotland,Wales,2019,-7,81.86,87.26,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
#On supprimme les colonnes Team et Opponent
df = df.drop("Team", axis='columns')
df = df.drop("Opponent", axis='columns')

df.head()

,Year,Diff,home_standing,away_standing,Team_0,Team_1,Team_2,Team_3,Team_4,Team_5,Opponent_0,Opponent_1,Opponent_2,Opponent_3,Opponent_4,Opponent_5
0,2014,8,80.19,73.13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2014,2,80.87,85.70,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2019,-5,77.34,87.26,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2020,7,84.24,81.21,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2020,42,79.77,70.88,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
#On replace la variable cible à la dernière colonne
diff=df.pop('Diff')
df.insert(15,'Diff',diff)
df.head()

,Year,home_standing,away_standing,Team_0,Team_1,Team_2,Team_3,Team_4,Team_5,Opponent_0,Opponent_1,Opponent_2,Opponent_3,Opponent_4,Opponent_5,Diff
0,2014,80.19,73.13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8
1,2014,80.87,85.70,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2
2,2019,77.34,87.26,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-5
3,2020,84.24,81.21,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7
4,2020,79.77,70.88,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,42


In [13]:
#On vérifie le type des données
df.dtypes

Year              object
home_standing    float64
away_standing    float64
Team_0           float64
Team_1           float64
Team_2           float64
Team_3           float64
Team_4           float64
Team_5           float64
Opponent_0       float64
Opponent_1       float64
Opponent_2       float64
Opponent_3       float64
Opponent_4       float64
Opponent_5       float64
Diff               int64
dtype: object

In [33]:
df['Year'] = df['Year'].astype(int)
df.dtypes


Year               int64
home_standing    float64
away_standing    float64
Team_0           float64
Team_1           float64
Team_2           float64
Team_3           float64
Team_4           float64
Team_5           float64
Opponent_0       float64
Opponent_1       float64
Opponent_2       float64
Opponent_3       float64
Opponent_4       float64
Opponent_5       float64
Diff               int64
dtype: object

In [15]:
#On associe les colonnes aux X et Y
X = df.loc[:, 'Year':'Opponent_5']
X = X.values #Transforme en arrays (matrice d'arrays)
print('Shape of X: ', X.shape)
y = df.loc[:, 'Diff']
y = y.values #Transforme en arrays (matrice d'arrays)
y = y.reshape(y.shape[0],1)
print('Shape of Y: ', y.shape)


Shape of X:  (255, 15)
Shape of Y:  (255, 1)


## STANDART SCALER

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Entrainement de modèles

RÉGRESSION LINÉAIRE

In [18]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()

In [19]:
LR.fit(X_train, y_train)

LinearRegression()

In [20]:
prediction = LR.predict(X_test)

In [21]:
from sklearn.metrics import mean_absolute_error
mae_score = mean_absolute_error(y_test, prediction)

In [22]:
print(mae_score)

10.523395290908553


RÉGRESSION LASSO

In [23]:
from sklearn.linear_model import Lasso
LA = Lasso()

In [24]:
LA.fit(X_train, y_train)

Lasso()

In [25]:
prediction = LA.predict(X_test)
mae_score = mean_absolute_error(y_test, prediction)
print(mae_score)

10.911094192552106


RÉGRESSION RIDGE

In [26]:
from sklearn.linear_model import Ridge
RI = Ridge()
RI.fit(X_train, y_train)

Ridge()

In [27]:
prediction = RI.predict(X_test)
mae_score = mean_absolute_error(y_test, prediction)
print(mae_score)

10.517984769474268


RÉGRESSION ELASTIC NET

In [28]:
from sklearn.linear_model import ElasticNet
EN = ElasticNet()
EN.fit(X_train, y_train)

ElasticNet()

In [29]:
prediction = EN.predict(X_test)
mae_score = mean_absolute_error(y_test, prediction)
print(mae_score)

11.086595721908061


RÉGRESSION RANDOM FOREST

In [30]:
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor(max_depth=3)
y_train = y_train.ravel()

In [31]:
RFR.fit(X_train, y_train)
prediction = RFR.predict(X_test)
mae_score = mean_absolute_error(y_test, prediction)
print(mae_score)

11.326328067095458
